# Project 1


## Description

## Data import

In [1]:
import dlc_practical_prologue as prologue

# We import 1000 pairs of digits for the training and the testing inputs, targets and classes
number_of_pairs = 1000 
train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets( 
    number_of_pairs)

# Quick verification of the sizes:
print('Size of train_input:', train_input.size())
print('Size of train_target:', test_target.size())
print('Size of train_classes:', test_classes.size())
print('Size of test_input:', test_input.size())
print('Size of test_target:', test_target.size())
print('Size of test_classes:', test_classes.size())



Size of train_input: torch.Size([1000, 2, 14, 14])
Size of train_target: torch.Size([1000])
Size of train_classes: torch.Size([1000, 2])
Size of test_input: torch.Size([1000, 2, 14, 14])
Size of test_target: torch.Size([1000])
Size of test_classes: torch.Size([1000, 2])


## CNN

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

In [3]:
class ConvNet(nn.Module):
    def __init__(self, nb_hidden=200, initial_layers=2, final_layers=2):
        super().__init__()
        self.conv1 = nn.Conv2d(initial_layers, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(256, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, final_layers)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2, stride=2))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        return x

## CNN with Weight Sharing

#### We define the same class wether we use auxiliary loss or not

In [4]:
class WeightSharing(nn.Module):
    def __init__(self, auxiliary_loss = False):
        super().__init__()
        self.sharedConvNet = ConvNet(initial_layers=1, final_layers=10)
        self.auxiliary_loss = auxiliary_loss

        self.fc1 = nn.Linear(20,100)
        self.fc2 = nn.Linear(100,20)
        self.fc3 = nn.Linear(20,2)
    
    def forward(self, x):
        image1 = x.narrow(1,0,1)
        image2 = x.narrow(1,1,1)

        output1 = self.sharedConvNet(image1)
        output2 = self.sharedConvNet(image2)
        outputCat = torch.cat((output1, output2), 1)

        outputCat = F.relu(self.fc1(outputCat))
        outputCat = F.relu(self.fc2(outputCat))
        outputCat = self.fc3(outputCat)

        
        if self.auxiliary_loss:
            return outputCat, output1, output2
        else:
            return outputCat

## Model training and number of errors

In [5]:
def train_model(model, train_input, train_target, mini_batch_size, nb_epochs = 50, lr = 1e-1, printAccLoss = False, auxiliary_loss = False, alpha = 0.3, beta = 0.5):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr = lr)

    for e in range(nb_epochs):
        acc_loss = 0

        for b in range(0, train_input.size(0), mini_batch_size):
            if auxiliary_loss:
                trainTargetImage1 = (train_classes.narrow(1,0,1)).squeeze()
                trainTargetImage2 = (train_classes.narrow(1,1,1)).squeeze()

                outputCat, output1, output2 = model(train_input.narrow(0, b, mini_batch_size))
                lossCat = criterion(outputCat, train_target.narrow(0, b, mini_batch_size))
                loss1 = criterion(output1, trainTargetImage1.narrow(0, b, mini_batch_size))
                loss2 = criterion(output2, trainTargetImage2.narrow(0, b, mini_batch_size))

                loss = alpha*loss1 + alpha*loss2 + beta*lossCat

            else:
                output = model(train_input.narrow(0, b, mini_batch_size))
                loss = criterion(output, train_target.narrow(0, b, mini_batch_size))

            acc_loss = acc_loss + loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


        if printAccLoss:
            print(e, acc_loss)

def compute_nb_errors(model, input, target, mini_batch_size, auxiliary_loss = False):
    nb_errors = 0

    for b in range(0, input.size(0), mini_batch_size):
        if auxiliary_loss:
            outputCat, _, _ = model(input.narrow(0, b, mini_batch_size))
            _, predicted_classes = outputCat.max(1)
            for k in range(mini_batch_size):
                if target[b + k] != predicted_classes[k]:
                    nb_errors = nb_errors + 1
        else:
            output = model(input.narrow(0, b, mini_batch_size))
            _, predicted_classes = output.max(1)
            for k in range(mini_batch_size):
                if target[b + k] != predicted_classes[k]:
                    nb_errors = nb_errors + 1


    return nb_errors

## Models testing

In [6]:
import time
mini_batch_size = 50
number_of_runs = 10

### CNN

In [7]:
for k in range(number_of_runs):
    train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(number_of_pairs)
    startTime = time.time()
    model = ConvNet()
    midTime = time.time()
    train_model(model, train_input, train_target, mini_batch_size)
    endTime = time.time()
    nb_test_errors = compute_nb_errors(model, test_input, test_target, mini_batch_size)
    print('test error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0), nb_test_errors, test_input.size(0)))
    print('Time for model:', str(midTime-startTime) + ', time for training:', endTime-midTime)

test error Net 47.60% 476/1000
Time for model: 0.02599930763244629, time for training: 4.4058678150177
test error Net 43.20% 432/1000
Time for model: 0.0020003318786621094, time for training: 4.6303136348724365
test error Net 54.30% 543/1000
Time for model: 0.0009992122650146484, time for training: 4.478524208068848
test error Net 42.50% 425/1000
Time for model: 0.0019989013671875, time for training: 4.165874719619751
test error Net 43.70% 437/1000
Time for model: 0.0010013580322265625, time for training: 4.406866788864136
test error Net 43.20% 432/1000
Time for model: 0.0020003318786621094, time for training: 4.577165126800537
test error Net 45.50% 455/1000
Time for model: 0.0009999275207519531, time for training: 4.325835227966309
test error Net 45.40% 454/1000
Time for model: 0.002000093460083008, time for training: 4.370460748672485
test error Net 46.40% 464/1000
Time for model: 0.002000570297241211, time for training: 4.178874492645264
test error Net 54.70% 547/1000
Time for model

### CNN with Weight Sharing

In [8]:
for k in range(number_of_runs):
    train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(number_of_pairs)
    startTime = time.time()
    model = WeightSharing()
    midTime = time.time()
    train_model(model, train_input, train_target, mini_batch_size)
    endTime = time.time()
    nb_test_errors = compute_nb_errors(model, test_input, test_target, mini_batch_size)
    print('test error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0), nb_test_errors, test_input.size(0)))
    print('Time for model:', str(midTime-startTime) + ', time for training:', endTime-midTime)

test error Net 55.60% 556/1000
Time for model: 0.0020008087158203125, time for training: 9.305521726608276
test error Net 54.70% 547/1000
Time for model: 0.003000497817993164, time for training: 9.900016784667969
test error Net 43.70% 437/1000
Time for model: 0.0029997825622558594, time for training: 10.271310091018677
test error Net 55.40% 554/1000
Time for model: 0.003000497817993164, time for training: 10.051294565200806
test error Net 54.30% 543/1000
Time for model: 0.0019998550415039062, time for training: 9.9918851852417
test error Net 52.40% 524/1000
Time for model: 0.0020003318786621094, time for training: 9.408778667449951
test error Net 55.60% 556/1000
Time for model: 0.002000570297241211, time for training: 9.712709665298462
test error Net 54.90% 549/1000
Time for model: 0.002000093460083008, time for training: 10.020086288452148
test error Net 52.20% 522/1000
Time for model: 0.0019991397857666016, time for training: 12.245321035385132
test error Net 55.60% 556/1000
Time for

### CNN with Weight Sharing and Auxiliary Loss

In [9]:
for k in range(number_of_runs):
    train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(number_of_pairs)
    startTime = time.time()
    model = WeightSharing(auxiliary_loss=True)
    midTime = time.time()
    train_model(model, train_input, train_target, mini_batch_size, auxiliary_loss=True)
    endTime = time.time()
    nb_test_errors = compute_nb_errors(model, test_input, test_target, mini_batch_size, auxiliary_loss=True)
    print('test error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0), nb_test_errors, test_input.size(0)))
    print('Time for model:', str(midTime-startTime) + ', time for training:', endTime-midTime)

KeyboardInterrupt: 